## Import

In [1]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

import pandas as pd
import numpy as np

## Scraping 624 pages

In [29]:
# empty dataframe

rental = pd.DataFrame(columns=['address','price','area', 'rooms', 'condition', 'year', 'floor',
             'elevator', 'heat', 'air_conditioner', 'furnished', 'accessible',
             'bathroom_toilet', 'view', 'balcony', 'mechanized', 'pet'])


In [18]:
# header for the request

headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}

In [31]:
# loop through pages to get the landing pages

for i in range(1,625):
    
    # website used for loop
    website = 'https://ingatlan.com/szukites/kiado+lakas+budapest?page=' + str(i)
    
    # request
    response = requests.get(website, headers=headers)
    
    # create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # create result container
    result_container = soup.select('div[class*="listing js-listing"]')
    
    # URL parts
    home_page = "https://ingatlan.com"
    
    landing_url = []
    
    # loop to get all landings
    for item in result_container:
        for link in item.find_all('a', {'class':'listing__thumbnail js-listing-active-area'}):
            landing_url.append(link.get('href'))

    # join urls
    url_joined = []

    for link in landing_url:
        url_joined.append(urllib.parse.urljoin(home_page, link))
    
    # looping throuh joined urls
    for link in url_joined:
        response = requests.get(link, headers=headers)

        # create soup object
        soup = BeautifulSoup(response.content, 'html.parser')

        # extract data

        price = soup.select('.parameterValues')[0].get_text().strip()
        area = soup.select('.parameterValues')[1].get_text().strip()
        rooms = soup.select('.parameterValues')[2].get_text().strip()
        address = soup.find('h1', {'class':'address'}).get_text().strip()
        condition = soup.select('.parameterValue')[0].get_text().strip()
        year = soup.select('.parameterValue')[1].get_text().strip()

        # Some pages don't display floor number of lodgings and it messes up the indexing

        if soup.select('.parameterName')[4].get_text().strip() == 'Emelet':

                floor = soup.select('.parameterValue')[4].get_text().strip()
                elevator = soup.select('.parameterValue')[6].get_text().strip()
                heat = soup.select('.parameterValue')[8].get_text().strip()
                air_conditioner = soup.select('.parameterValue')[9].get_text().strip()
                furnished = soup.select('.parameterValue')[10].get_text().strip()
                accessible = soup.select('.parameterValue')[14].get_text().strip()
                bathroom_toilet = soup.select('.parameterValue')[15].get_text().strip()
                view = soup.select('.parameterValue')[17].get_text().strip()
                balcony = soup.select('.parameterValue')[18].get_text().strip()
                mechanized = soup.select('.parameterValue')[21].get_text().strip()
                pet = soup.select('.parameterValue')[22].get_text().strip()

        else:
                floor = 'N/A'
                elevator = soup.select('.parameterValue')[4].get_text().strip()
                heat = soup.select('.parameterValue')[6].get_text().strip()
                air_conditioner = soup.select('.parameterValue')[7].get_text().strip()
                furnished = soup.select('.parameterValue')[8].get_text().strip()
                accessible = soup.select('.parameterValue')[12].get_text().strip()
                bathroom_toilet = soup.select('.parameterValue')[13].get_text().strip()
                view = soup.select('.parameterValue')[15].get_text().strip()
                balcony = soup.select('.parameterValue')[16].get_text().strip()
                mechanized = soup.select('.parameterValue')[19].get_text().strip()
                pet = soup.select('.parameterValue')[20].get_text().strip()
            
        # append data to dataframe
        rental = rental.append({'address':address,'price':price, 'area':area, 'rooms':rooms, 'condition':condition, 'year': year, 'floor':floor,
                 'elevator':elevator, 'heat':heat, 'air_conditioner':air_conditioner, 'furnished':furnished, 'accessible':accessible,
                 'bathroom_toilet':bathroom_toilet, 'view':view, 'balcony':balcony, 'mechanized':mechanized, 'pet':pet}, ignore_index=True)

In [32]:
rental

,address,price,area,rooms,condition,year,floor,elevator,heat,air_conditioner,furnished,accessible,bathroom_toilet,view,balcony,mechanized,pet
0,"VIII. kerület, Orczy út 46-48",105 ezer Ft,25 m²,1,új építésű,2021,1,nincs megadva,házközponti egyedi méréssel,van,igen,nincs megadva,egy helyiségben,nincs megadva,nincs megadva,igen,nincs megadva
1,"VIII. kerület, Orczy út 46-48",120 ezer Ft,44 m²,1,felújított,nincs megadva,1,nincs,házközponti egyedi méréssel,van,megegyezés szerint,nincs megadva,egy helyiségben,udvari,nincs megadva,igen,nem hozható
2,"V. kerület, Városház utca",176 ezer Ft,36 m²,1,felújított,nincs megadva,4,van,házközponti,van,nem,nincs megadva,egy helyiségben,nincs megadva,nincs megadva,nem,nem hozható
3,"V. kerület, Semmelweis utca 11.",90 ezer Ft,20 m²,1,újszerű,2016,N/A,van,házközponti,van,igen,igen,egy helyiségben,panorámás,nincs megadva,igen,nem hozható
4,"V. kerület, Semmelweis utca 11.",56 ezer Ft,46 m²,1,újszerű,2016,N/A,van,házközponti,van,igen,igen,egy helyiségben,panorámás,nincs megadva,igen,nem hozható
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10323,"V. kerület, Só utca","1,5 ezer €",108 m²,3,felújított,nincs megadva,3,nincs,gáz (cirko),nincs megadva,igen,nincs megadva,külön és egyben is,utcai,nincs megadva,nincs megadva,nincs megadva
10324,"VI. kerület, Hajós utca","1,2 ezer €",90 m²,3,újszerű,1950 előtt,1,nincs,gáz (cirko),nincs,igen,nincs megadva,külön és egyben is,panorámás,nincs megadva,igen,nincs megadva
10325,"II. kerület, Pasaréti út","2,3 ezer €",220 m²,4,felújított,1950 előtt,2,nincs,gáz (cirko),van,megegyezés szerint,nem,külön és egyben is,panorámás,10 m²,igen,hozható
10326,"VI. kerület, Lázár utca","1,2 ezer €",139 m²,4,felújított,1950 előtt,3,van,gáz (cirko),van,nem,nincs megadva,külön és egyben is,utcai,1 m²,igen,nem hozható
